# Web-Project

## Propósito del proyecto

El principal propósito de este proyecto es obtener una lista de citas célebres, junto con su autor y con las caractéristicas de la cita, como el tipo de cita que es. Esta información la obtendres haciedno web scraping de la web "https://www.azquotes.com/top_quotes.html", la cual contiene exactamente 1000 citas célebres.

A partir de ahí la idea es utilizar la API de Wikipedia para obtener información respecto al autor de cada cita. Y una vez la haya conseguido, juntar ambas bases de datos para hacer una más grande y enriquecida.

## Código

A continuación importo las librerías que vaya considerando necesarias para trabajar con la extracción de los datos.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from lxml import html
import urllib.request
from urllib.request import urlopen
import re
import wikipedia
import itertools
import json
from googletrans import Translator
import tweepy

# from pprint import pprint
# from lxml.html import fromstring
# import random
# import scrapy

## Web scraping

En la celda de debajo he escrito el código con el que obtengo las citas con sus respectivo autores y tags, rotando por cada una de las 10 páginas de la web a través del loop for. Una vez tengo toda la información que quiero, la junto utilizando un diccionario, y la convierto en un DataFrame de 4 columnas, incluyendo el índice, el cual he formateado para que empiece por 1, en vez de 0.

In [2]:
list_of_quotes = []
list_of_authors = []
list_of_tags = []
url = 'https://www.azquotes.com/top_quotes.html?p=%s'

for i in range(1, 11):
    html = requests.get(url % i).content
    soup = BeautifulSoup(html, 'lxml')

    list_of_quotes.append([element.text.replace("\n", "") for element in soup.find_all('a',{'class':'title'})])
    merged_quotes = list(itertools.chain.from_iterable(list_of_quotes))
        
    list_of_authors.append([element.text.replace("\n", "").replace("Xunzi", "Xun Kuang").replace("John Lubbock", "John Lubbock, 1st Baron Avebury").replace("Drake", "Drake (musician)").replace("Charles Simmons", "Charles Simmons (author)").replace("James Allen", "James Allen (author)").replace("John Green", "John Green (author)").replace("Unknown", "Unknown (mathematics)") for element in soup.find_all('div',{'class':'author'})])
    merged_authors = list(itertools.chain.from_iterable(list_of_authors))
        
    list_of_tags.append([element.text.replace("\n", "") for element in soup.find_all('div',{'class':'tags'})])
    merged_tags = list(itertools.chain.from_iterable(list_of_tags))
        
data = pd.DataFrame({'Quote':merged_quotes,'Author':merged_authors,'Tags':merged_tags})
data.index = np.arange(1, len(data)+1)


## Wikipedia API

A continuación, utilizo la API de Wikipedia para ir rotando por la lista de autores de cada una de las citas para obtener un breve resumen con la biografía del autor y el link con el resto de la información en Wikipedia. Según sacamos la información, la incluimos en una lista que posteriormente convertiremos a DataFrame.

In [3]:
S = requests.Session()
URL = "https://en.wikipedia.org/w/api.php"

info = []
info2 = []
for author in merged_authors:
    PARAMS = {
        'action':"opensearch",
        'search':author,
        'limit': 0,
        'namespace':0,
        'profile':"normal",
        'format':"json"
        }
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    DATA2 = ''.join(DATA[2])
    DATA3 = ''.join(DATA[3])
    info.append(DATA2)
    info2.append(DATA3)
    
        
info = pd.DataFrame({'Authors_info':info, 'Wikipedia_link':info2})
info.index = np.arange(1, len(info)+1)

Aunque también encontré una seguna forma de hacerlo a través de comandos, que daba un resumen del autor un poco más largo en el que puedo especificar el número de frases a incluir, pero que no aportaba el link que finalmente también he incluido en la tabla.

In [4]:
#def quote(url):
#    list_of_quotes = []
#    list_of_authors = []
#    list_of_tags = []
#    authors_info = []
#    for i in range(1, 11):
#        html = requests.get(url % i).content
#        soup = BeautifulSoup(html, 'lxml')
#
#        list_of_quotes.append([element.text.replace("\n", "") for element in soup.find_all('a',{'class':'title'})])
#        merged_quotes = list(itertools.chain.from_iterable(list_of_quotes))
#        
#        list_of_authors.append([element.text.replace("\n", "").replace("Xunzi", "Xun Kuang").replace("John Lubbock", "John Lubbock, 1st Baron Avebury").replace("Drake", "Drake (musician)").replace("Charles Simmons", "Charles Simmons (author)").replace("James Allen", "James Allen (author)").replace("John Green", "John Green (author)").replace("Unknown", "Unknown (mathematics)") for element in soup.find_all('div',{'class':'author'})])
#        merged_authors = list(itertools.chain.from_iterable(list_of_authors))
#        
#        list_of_tags.append([element.text.replace("\n", "") for element in soup.find_all('div',{'class':'tags'})])
#        merged_tags = list(itertools.chain.from_iterable(list_of_tags))
#    
#    for author in merged_authors:
#        authors_info.append(wikipedia.summary(author, sentences=3))
#        
#    data = pd.DataFrame({'Quote':merged_quotes,'Author':merged_authors,'Tags':merged_tags,'Author_info':authors_info})
#    data.index = np.arange(1, len(data)+1)
#
#    return data
#    
#data = quote('https://www.azquotes.com/top_quotes.html?p=%s')

Ahora juntamos ambos datasets para crear una base de datos más grande y más enriquecida. Como podemos observar, la columna de la derecha corresponde con la información del autor extraida de la Wikipedia.

In [5]:
final = pd.concat([data, info], axis = 1, )
final.head()

,Quote,Author,Tags,Authors_info,Wikipedia_link
1,The essence of strategy is choosing what not t...,Michael Porter,"Essence, Deep Thought, Transcendentalism","Michael Eugene Porter (born May 23, 1947) is ...",https://en.wikipedia.org/wiki/Michael_Porter
2,Death is something inevitable. When a man has ...,Nelson Mandela,"Inspirational, Motivational, Death",Nelson Rolihlahla Mandela (; Xhosa: [xolíɬaɬa ...,https://en.wikipedia.org/wiki/Nelson_Mandela
3,"With or without religion, you would have good ...",Steven Weinberg,"God, Religious, Atheist","Steven Weinberg (; born May 3, 1933) is an Am...",https://en.wikipedia.org/wiki/Steven_Weinberg
4,A person who never made a mistake never tried ...,Albert Einstein,"Inspirational, Motivational, Change",Albert Einstein (; German: [ˈalbɛɐ̯t ˈʔaɪnʃtaɪ...,https://en.wikipedia.org/wiki/Albert_Einstein
5,"My mission in life is not merely to survive, b...",Maya Angelou,"Inspirational, Life, Inspiring",Maya Angelou ( (listen); born Marguerite Annie...,https://en.wikipedia.org/wiki/Maya_Angelou


## Twitter API

He decidido utilizar una segunda API para saber cómo de populares son en la actualidad los autores de las citas, obteniendo tweets clasificados como populares que contengan el nombre del autor. El problema es que he alcanzado el límite de peticiones.

La fórmula funcionaba porque la he utilizado para obtener el número de tweets populares sobre Alex Txikon (un escalador español), Himalaya, Esperanza Aguirre y Colón, y me daba cantidades de alrededor de 4 y 25, bastante razonables.

Dado que no puedo hacer la petición para todo mi DataFrame, dejo puesta la fórmula completa.

In [6]:
API_KEY = "RMWYgf9Sqeg5KHXZBXIDObEYr"
API_SECRET = "7Bn4txTaTp5RW3mIVWJ5E8aIw1k4KvUzkd8HWGvzcH1TwqM4Qk"
ACCESS_TOKEN = "385891004-NSZjAh60rbYBC1yb4VxTC4uqyabLSdT29hBFhEMw"
ACCESS_TOKEN_SECRET = "7PRcEc2X3ZZ44q45u8DQ6pjvlsxAxGNYNGKrd06XDeLAu"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [8]:
lista_tweets = []

for author in merged_authors:
    results = api.search(q=author, result_type = "recent", count = 100)
    lista_tweets.append(len(results))


RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [12]:
lista_tweets = pd.DataFrame({'Popular_tweets_count':lista_tweets})
lista_tweets.index = np.arange(1, len(lista_tweets)+1)
lista_tweets.head()

ValueError: If using all scalar values, you must pass an index

In [14]:
final2 = pd.concat([final, lista_tweets], axis = 1, )
final2.head()

,Quote,Author,Tags,Authors_info,Wikipedia_link,Popular_tweets_count
1,The essence of strategy is choosing what not t...,Michael Porter,"Essence, Deep Thought, Transcendentalism","Michael Eugene Porter (born May 23, 1947) is ...",https://en.wikipedia.org/wiki/Michael_Porter,100.0
2,Death is something inevitable. When a man has ...,Nelson Mandela,"Inspirational, Motivational, Death",Nelson Rolihlahla Mandela (; Xhosa: [xolíɬaɬa ...,https://en.wikipedia.org/wiki/Nelson_Mandela,100.0
3,"With or without religion, you would have good ...",Steven Weinberg,"God, Religious, Atheist","Steven Weinberg (; born May 3, 1933) is an Am...",https://en.wikipedia.org/wiki/Steven_Weinberg,75.0
4,A person who never made a mistake never tried ...,Albert Einstein,"Inspirational, Motivational, Change",Albert Einstein (; German: [ˈalbɛɐ̯t ˈʔaɪnʃtaɪ...,https://en.wikipedia.org/wiki/Albert_Einstein,100.0
5,"My mission in life is not merely to survive, b...",Maya Angelou,"Inspirational, Life, Inspiring",Maya Angelou ( (listen); born Marguerite Annie...,https://en.wikipedia.org/wiki/Maya_Angelou,100.0


## Google Translate API

Con la siguiente API lo que pretendo es traducir cada una de las citas al español. Aunque el funcionamiento de esta API se basa en funciones y no en peticiones que se hacen a una URL como tal como en el caso de la Wikipedia.

Para ello, y de forma similar a la anterior API, trabajo con un bucle que para cada cita me haga la traducción del inglés al español. Y como antes, inserto el resultado en un DataFrame que posteriormente fusiono con las dos bases de datos anteriores.

In [15]:
tranlations_list = []
for quote in merged_quotes:
    translator = Translator()
    translations = translator.translate(quote, dest='es', src='en')
    tranlations_list.append(translations)


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
tranlations_list = pd.DataFrame({'Quote_in_spanish':tranlations_list})
tranlations_list.index = np.arange(1, len(tranlations_list)+1)
tranlations_list.head(35)

In [ ]:
final3 = pd.concat([final2, tranlations_list], axis = 1, )
final3.head()